#Using Embalance Learning(Text Augmentation) on Imbalance data to balance and perform CNN model binary classifier

In [1]:
pip install nlpaug

     |████████████████████████████████| 410 kB 10.4 MB/s 


In [2]:

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [5]:
#setup="from __main__ import gpu"

In [6]:
import pandas as pd
import csv
import re
import nlpaug
import nlpaug.augmenter.word as naw

In [7]:
import os
import math
import gc
import pickle
from keras.initializers import Constant

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import numpy as np
from csv import writer
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics

#for Smote 
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot

In [8]:
pip install numpy nltk gensim textblob googletrans 

     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 1.3 MB 15.9 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 4.2 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=81e71ca608dfd73ed873bea8fc15db61e2698734b5b606047fe836eeba6f55fd
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [9]:
pip install googletrans==3.1.0a0

  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=02946e90751c666a0588603a9e92fc9e138a51d4067147fe85df45a7ce7ef926
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [10]:
pip install textaugment

In [11]:
from textaugment import Word2vec
from textaugment import Wordnet
from textaugment import Translate


In [12]:
from textaugment import EDA


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
df= pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/TextAugmentation/Data/raw_yelp_review_data.csv")

In [15]:
df.head(5)

,coffee_shop_name,full_review_text,star_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating


In [16]:
#df['full_review_text']

In [17]:

def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    no_digits=[]
    for i in in_str:
      if not i.isdigit():
        no_digits.append(i)
        # Now join all elements of the list with '', 
        # which puts all of the characters together. 
    in_str = ''.join(no_digits)
    return in_str.strip().lower()


df['text'] = df['full_review_text'].apply(clean_str)


In [18]:
df.head(5)

,coffee_shop_name,full_review_text,star_rating,text
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating,checkin love love loved the atmosphere every c...
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating,listed in date night austin ambiance in austin...
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating,checkin listed in brunch spots i loved the ecl...
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating,very cool decor good drinks nice seating howe...
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating,checkin they are located within the northcross...


In [19]:
sentence_length=0
for line in df['text']:
  max=0
  for i in line:
    max+=1
  
  if(sentence_length<max):
    sentence_length=max


In [20]:
print(sentence_length)

4884


#CNN Model


In [21]:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

In [22]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline


In [23]:
#data=df

In [24]:
max_features = 20000 # this is the number of words we care about
sequence_length=sentence_length #52


In [25]:
#trace only rating from the star rating and store in the column 'rating'
#create two classes 1 means positive and 0 means negative review

raiting=[]
bool_raiting=[]
for line1 in df['star_rating']:

  x=int(line1[1:2])
  raiting.append(x)
  if x>3:
    bool_raiting.append(1)
  else:
    bool_raiting.append(0)
    
df['bool_raiting']= bool_raiting
df['raiting']=raiting
  #count+=1
pos=0
neg=0
for i in df['bool_raiting']:
  if i==1:
    pos+=1
  else:
    neg+=1
print("Total size of dataset = ",len(df['text']))
print("Positive(1) classes = ",pos)
print("Negative(0) classes = ",neg)
#sst2_data=df


Total size of dataset =  7616
Positive(1) classes =  6140
Negative(0) classes =  1476


In [26]:
#sst2_data['Sentiment'] = sst2_data['Sentiment'].apply(merge_sentiments)
def embedding(text,bool_raiting,sequence_length):
  tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
  tokenizer.fit_on_texts(text.values)

  sst2_X = tokenizer.texts_to_sequences(text)
  sst2_X = pad_sequences(sst2_X, sequence_length)

  sst2_y = bool_raiting  #sst2_data['bool_raiting'].values

  embeddings_index = {}
  f = open('/content/drive/MyDrive/Colab_Notebooks/TextAugmentation/glove-global-vectors-for-word-representation/glove.6B.200d.txt')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  word_index = tokenizer.word_index

  print('Found %s unique tokens.' % len(word_index))

  num_words = min(max_features, len(word_index)) + 1
  print(num_words)
  embedding_dim = 200 
  num_filters = 100
  # first create a matrix of zeros, this is our embedding matrix
  embedding_matrix = np.zeros((num_words, embedding_dim))

  # for each word in out tokenizer lets try to find that work in our w2v model
  for word, i in word_index.items():
      if i > max_features:
          continue
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # we found the word - add that words vector to the matrix
          embedding_matrix[i] = embedding_vector
      else:
          # doesn't exist, assign a random vector
          embedding_matrix[i] = np.random.randn(embedding_dim)
  return (sst2_X,sst2_y,embedding_matrix,num_words)
#sst2_X_train, sst2_X_test, sst2_y_train, sst2_y_test = train_test_split(sst2_X, sst2_y, test_size=0.1)

In [27]:
def cnn_model(num_words,embedding_matrix,sequence_length):

  embedding_dim = 200 
  num_filters = 100

  inputs_4 = Input(shape=(sequence_length,), dtype='int32')
  embedding_layer_4 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_4)

  reshape_4 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_4)

  conv_0_4 = Conv2D(num_filters, kernel_size=(3, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
  conv_1_4 = Conv2D(num_filters, kernel_size=(4, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
  conv_2_4 = Conv2D(num_filters, kernel_size=(5, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)

  maxpool_0_4 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_4)
  maxpool_1_4 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_4)
  maxpool_2_4 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_4)

  concatenated_tensor_4 = Concatenate(axis=1)([maxpool_0_4, maxpool_1_4, maxpool_2_4])
  flatten_4 = Flatten()(concatenated_tensor_4)

  dropout_4 = Dropout(0.5)(flatten_4)
  # note the different activation
  output_4 = Dense(units=1, activation='sigmoid')(dropout_4)

  model = Model(inputs=inputs_4, outputs=output_4)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  return model

In [28]:
def predict_result(X_test,y_test, name,size_of_data,model):
  #perform prediction
  total=len(X_test)
  
  p_hat = model.predict(X_test)
  y_hat = np.where(p_hat > 0.5, 1, 0)
  #print(y_hat)
  #tn, fp, fn, tp = confusion_matrix(y_test,y_hat).ravel()
  result=metrics.classification_report(y_test,y_hat, labels=[0,1])
  cm=metrics.confusion_matrix(y_test,y_hat, labels=[0,1])
  
  print("Confusion matrix: \n",cm)

  return result
  

In [29]:
def my_model(emb_input, output_bool,model):
  #X= train_input_ids
  X=emb_input
  #y = pd.get_dummies(data['Sentiment']).values
  y=output_bool
  #X_np= np.array(X,dtype='int32')
  X_np= np.array(X,dtype='float32')

  
  X_train, X_test, y_train, y_test = train_test_split(X_np, y, test_size=0.1,stratify=y)
  
  print("test set size " + str(len(X_test)))
  print("Train set size " + str(len(X_train)))

  batch_size=32
  history_4 = model.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1)   #, validation_split=0.2)
  #plot_result(history)
  return (X_test, y_test)





In [30]:

def shuffle_data(shuf):
  shuffled = shuf.sample(frac=1)
  return shuffled

#Dealing with minority class

In [31]:
pos=0
neg=0

for i in df['bool_raiting']:
  if i==1:
    pos+=1
  else:
    neg+=1
print("Total size of dataset = ",len(df['text']))
print("Positive(1) classes = ",pos)
print("Negative(0) classes = ",neg)
if(neg<pos):
  minor_class=0
else:
  minor_class=1


Total size of dataset =  7616
Positive(1) classes =  6140
Negative(0) classes =  1476


In [32]:
minor_class

0

In [33]:
columns=['text','bool_raiting']

df_1 = pd.DataFrame(columns=columns)
df_1 = df_1.fillna(0)

In [34]:
#separate the dataset of minor class to perform data balancing
count=0
#i=0
text_1=[]
bool_raiting1=[]  #
bool_raiting=df['bool_raiting']
text=df['text']
for i in range(len(df['text'])):
  if int(bool_raiting[i])==minor_class:
    #print("text ",i," =",text[i])
    #print(df['bool_raiting'][i])
    #df_1['bool_raiting'][i]=0
    bool_raiting1.append(minor_class)
    text_1.append(text[i])
    count+=1
df_1['text']=text_1
df_1['bool_raiting']=bool_raiting1
#error to show there is a problem
print(df_1)

                                                   text  bool_raiting
0     very cool decor good drinks nice seating  howe...             0
1     checkin ok lets try this approach  pros music ...             0
2     the store has a decoration but honestly the co...             0
3     checkin listed in   the third time is a charm ...             0
4     okay so after visiting my friends in la and tr...             0
...                                                 ...           ...
1471  i should probably give this spot  stars but th...             0
1472  its a lovely little tea and pastry shop but th...             0
1473  so today i tried this place i have been in aus...             0
1474  great service with unimaginative and uneventfu...             0
1475  the food is just average the booths didnt feel...             0

[1476 rows x 2 columns]


In [35]:
if pos>neg:
  difference_between_classes= pos-neg
  div=neg
else:
  difference_between_classes=neg-pos
  div=pos
print(difference_between_classes)

4664


In [36]:
num_word_sy=2
num_synth_sentence=difference_between_classes/div
#num_synth_sentence=3.55
print(num_synth_sentence)
round_num_synth_sentence=int(round(num_synth_sentence, 0))
print(round_num_synth_sentence)
print("total number of synthetic sentence will be created ",round_num_synth_sentence*neg)

3.1598915989159893
3
total number of synthetic sentence will be created  4428


#Original Data 

In [ ]:
df1=df
shuffled1= shuffle_data(df1)

In [ ]:
#Original Data
X1,y1,embedding_matrix1,num_words1 =embedding(shuffled1['text'],shuffled1['bool_raiting'].values,sequence_length)#original data

Found 400000 word vectors.
Found 19862 unique tokens.
19863


In [ ]:
columns=['embedding','bool_raiting']
original_df = pd.DataFrame(columns=columns)
original_df = original_df.fillna(0)

In [ ]:
model1= cnn_model(num_words1,embedding_matrix1,sequence_length)

X_test1, y_test1=my_model(X1, y1,model1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4884)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 4884, 200)    3972600     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 4884, 200, 1  0           ['embedding[0][0]']              
                                )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 4882, 1, 100  60100       ['reshape[0][0]']            

In [ ]:
size_of_data1= len(df['text'])
original_result =predict_result(X_test1,y_test1, 'Original',size_of_data1,model1)
print(original_result)
#accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))

Confusion matrix: 
 [[ 30 118]
 [ 10 604]]
              precision    recall  f1-score   support

           0       0.75      0.20      0.32       148
           1       0.84      0.98      0.90       614

    accuracy                           0.83       762
   macro avg       0.79      0.59      0.61       762
weighted avg       0.82      0.83      0.79       762



#Synthetic data using Text Augmentation

#EmbeddingAugmenter 

WordNet-based augmentation

In [ ]:
v = True # enable verbs augmentation. By default is True.
n = False # enable nouns augmentation. By default is False.
runs = 1 # number of times to augment a sentence. By default is 1.
p = 0.5 # The probability of success of an individual trial. (0.1<p<1.0), default is 0.5. Used by Geometric distribution to selects words from a sentence.
t = Wordnet(v=True ,n=True, p=0.5,runs=2)
wordnet_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    wordnet_text.append(t.augment(line))
print(len(wordnet_text))
#print(wordnet_text)
#In the afternoon, Joseph is going to town.

100%|██████████| 1476/1476 [00:25<00:00, 58.92it/s]

4428


In [ ]:
wordnet_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
wordnet_text_combine= df['text'].tolist()
wordnet_bool_raiting=[]
for i in df['bool_raiting']:
  wordnet_bool_raiting.append(i)
print(wordnet_bool_raiting)

for i in wordnet_text:
  wordnet_text_combine.append(i)
  wordnet_bool_raiting.append(minor_class)
  #count+=1


[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
columns=['text','bool_raiting']
df_wordnet_synth = pd.DataFrame(columns=columns)
df_wordnet_synth = df_wordnet_synth.fillna(0)
df_wordnet_synth['text']=wordnet_text_combine
df_wordnet_synth['bool_raiting']=wordnet_bool_raiting

In [ ]:
df_wordnet_synth.head(5)

,text,bool_raiting
0,checkin love love loved the atmosphere every c...,1
1,listed in date night austin ambiance in austin...,1
2,checkin listed in brunch spots i loved the ecl...,1
3,very cool decor good drinks nice seating howe...,0
4,checkin they are located within the northcross...,1


In [ ]:
df_wordnet_synth=shuffle_data(df_wordnet_synth)

In [ ]:
#Original Data
X_wn,y_wn,embedding_matrix_wn,num_words_wn =embedding(df_wordnet_synth['text'],df_wordnet_synth['bool_raiting'].values,sequence_length)#original data

Found 400000 word vectors.
Found 23013 unique tokens.
20001


In [ ]:
model_wn=cnn_model(num_words_wn,embedding_matrix_wn,sequence_length)
X_test_wn, y_test_wn=my_model(X_wn, y_wn,model_wn)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4884)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 4884, 200)    4000200     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 4884, 200, 1  0           ['embedding[0][0]']              
                                )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 4882, 1, 100  60100       ['reshape[0][0]']            

In [ ]:
size_of_data_wn= len(X_wn)
Wordnet_result= predict_result(X_test_wn,y_test_wn, 'wordnet',size_of_data_wn,model_wn)

print(Wordnet_result)


Confusion matrix: 
 [[547  44]
 [ 62 552]]
              precision    recall  f1-score   support

           0       0.90      0.93      0.91       591
           1       0.93      0.90      0.91       614

    accuracy                           0.91      1205
   macro avg       0.91      0.91      0.91      1205
weighted avg       0.91      0.91      0.91      1205



RTT-based augmentation

In [ ]:
src = "en" # source language of the sentence
to = "fr" # target language
from textaugment import Translate
rtt = Translate(src="en", to="fr")
rtt_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rtt_text.append(rtt.augment(line))
print(len(rtt_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 1476/1476 [00:58<00:00, 25.26it/s]

4428


In [ ]:
rtt_text

["very cool decoration good drink good seat but almost everything is too expensive for almond milk ounces in a cornoto without wifi music is a bit noisy and the mixture is strange pleasant french oldies followed by a strong techno several several tables were dirty when we got to the service is average, it's like a coffee stroke in los angeles that is out of words",
 "check ok, let's try this pro pros music seisting general decree approach conspiracy atmosphere ipad order im a crotchety old man kids limited coffee menu or menu in general relatively expensive food is pretty but dull, i'm still sitting here because i type this and i am less than delighted with the delight of the experience unfortunately the space and the atmosphere are great but not familiar or intimate enough to beat a place like a real joe in the street, it seems that they are banking on the atmosphere and fail the rest of the path the artsydesign crowd having a collaborative meeting the staff is great, but the ipad con

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rtt_text_combine= df['text'].tolist()
rtt_bool_raiting=[]
for i in df['bool_raiting']:
  rtt_bool_raiting.append(i)
print(rtt_bool_raiting)

for i in rtt_text:
  rtt_text_combine.append(i)
  rtt_bool_raiting.append(minor_class)
  #count+=1


[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
columns=['text','bool_raiting']
df_rtt_synth = pd.DataFrame(columns=columns)
df_rtt_synth = df_rtt_synth.fillna(0)
df_rtt_synth['text']=rtt_text_combine
df_rtt_synth['bool_raiting']=rtt_bool_raiting

In [ ]:
df_rtt_synth.head(5)

,text,bool_raiting
0,checkin love love loved the atmosphere every c...,1
1,listed in date night austin ambiance in austin...,1
2,checkin listed in brunch spots i loved the ecl...,1
3,very cool decor good drinks nice seating howe...,0
4,checkin they are located within the northcross...,1


In [ ]:
df_rtt_synth=shuffle_data(df_rtt_synth)

In [ ]:
#Original Data
X_rtt,y_rtt,embedding_matrix_rtt,num_words_rtt =embedding(df_rtt_synth['text'],df_rtt_synth['bool_raiting'].values,sequence_length)#original data

Found 400000 word vectors.
Found 21477 unique tokens.
20001


In [ ]:
model_rtt=cnn_model(num_words_rtt,embedding_matrix_rtt,sequence_length)
X_test_rtt, y_test_rtt=my_model(X_rtt, y_rtt,model_rtt)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4884)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 4884, 200)    4000200     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 4884, 200, 1  0           ['embedding[0][0]']              
                                )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 4882, 1, 100  60100       ['reshape[0][0]']            

In [ ]:
size_of_data_rtt= len(X_rtt)
rtt_result =predict_result(X_test_rtt,y_test_rtt, 'RTT Based',size_of_data_rtt,model_rtt)
print(rtt_result)


#EDA: Easy data augmentation techniques for boosting performance on text classification tasks

Synonym Replacement

In [ ]:
sr_text=[]
sr = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    sr_text.append(sr.synonym_replacement(line,2))
print(len(sr_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 1476/1476 [00:00<00:00, 2009.41it/s]

4428


In [ ]:
sr_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
sr_text_combine= df['text'].tolist()
sr_bool_raiting=[]
for i in df['bool_raiting']:
  sr_bool_raiting.append(i)
print(sr_bool_raiting)

for i in sr_text:
  sr_text_combine.append(i)
  sr_bool_raiting.append(minor_class)
  #count+=1


[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
columns=['text','bool_raiting']
df_sr_synth = pd.DataFrame(columns=columns)
df_sr_synth = df_sr_synth.fillna(0)
df_sr_synth['text']=sr_text_combine
df_sr_synth['bool_raiting']=sr_bool_raiting

In [ ]:
df_sr_synth.head(5)

,text,bool_raiting
0,checkin love love loved the atmosphere every c...,1
1,listed in date night austin ambiance in austin...,1
2,checkin listed in brunch spots i loved the ecl...,1
3,very cool decor good drinks nice seating howe...,0
4,checkin they are located within the northcross...,1


In [ ]:
df_sr_synth=shuffle_data(df_sr_synth)

In [ ]:
#Original Data
X_sr,y_sr,embedding_matrix_sr,num_words_sr =embedding(df_sr_synth['text'],df_sr_synth['bool_raiting'].values,sequence_length)#original data

Found 400000 word vectors.
Found 21589 unique tokens.
20001


In [ ]:
model_sr=cnn_model(num_words_sr,embedding_matrix_sr,sequence_length)
X_test_sr, y_test_sr=my_model(X_sr, y_sr,model_sr)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 4884)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 4884, 200)    4000200     ['input_2[0][0]']                
                                                                                                  
 reshape_1 (Reshape)            (None, 4884, 200, 1  0           ['embedding_1[0][0]']            
                                )                                                                 
                                                                                                  
 conv2d_3 (Conv2D)              (None, 4882, 1, 100  60100       ['reshape_1[0][0]']        

In [ ]:
size_of_data_sr= len(X_sr)
sr_result =predict_result(X_test_sr,y_test_sr, 'Synonyms replacement',size_of_data_sr,model_sr)
print(sr_result)


Confusion matrix: 
 [[410 181]
 [ 19 595]]
              precision    recall  f1-score   support

           0       0.96      0.69      0.80       591
           1       0.77      0.97      0.86       614

    accuracy                           0.83      1205
   macro avg       0.86      0.83      0.83      1205
weighted avg       0.86      0.83      0.83      1205



Random Deletion

In [ ]:
rd = EDA()
rd_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rd_text.append(rd.random_deletion(line, p=0.2))
print(len(rd_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

In [ ]:
rd_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rd_text_combine= df['text'].tolist()
rd_bool_raiting=[]
for i in df['bool_raiting']:
  rd_bool_raiting.append(i)
print(rd_bool_raiting)

for i in rd_text:
  rd_text_combine.append(i)
  rd_bool_raiting.append(minor_class)
  #count+=1


In [ ]:
columns=['text','bool_raiting']
df_rd_synth = pd.DataFrame(columns=columns)
df_rd_synth = df_rd_synth.fillna(0)
df_rd_synth['text']=rd_text_combine
df_rd_synth['bool_raiting']=rd_bool_raiting

In [ ]:
df_rd_synth.head(5)

In [ ]:
df_rd_synth=shuffle_data(df_rd_synth)

In [ ]:
#Original Data
X_rd,y_rd,embedding_matrix_rd,num_words_rd =embedding(df_rd_synth['text'],df_rd_synth['bool_raiting'].values,sequence_length)#original data

In [ ]:
model_rd=cnn_model(num_words_rd,embedding_matrix_rd,sequence_length)
X_test_rd, y_test_rd=my_model(X_rd, y_rd,model_rd)

In [ ]:
size_of_data_rd= len(X_rd)
rd_result =predict_result(X_test_rd,y_test_rd, 'Random Deletion',size_of_data_rd,model_rd)
print(rd_result)


Random Swap

In [ ]:
rs_text=[]
rs = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rs_text.append(rs.random_swap(line))
print(len(rs_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

In [ ]:
rs_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rs_text_combine= df['text'].tolist()
rs_bool_raiting=[]
for i in df['bool_raiting']:
  rs_bool_raiting.append(i)
print(rs_bool_raiting)

for i in rs_text:
  rs_text_combine.append(i)
  rs_bool_raiting.append(minor_class)
  #count+=1


In [ ]:
columns=['text','bool_raiting']
df_rs_synth = pd.DataFrame(columns=columns)
df_rs_synth = df_rs_synth.fillna(0)
df_rs_synth['text']=rs_text_combine
df_rs_synth['bool_raiting']=rs_bool_raiting

In [ ]:
df_rs_synth.head(5)

In [ ]:
df_rs_synth=shuffle_data(df_rs_synth)

In [ ]:
#Original Data
X_rs,y_rs,embedding_matrix_rs,num_words_rs =embedding(df_rs_synth['text'],df_rs_synth['bool_raiting'].values,sequence_length)#original data

In [ ]:
model_rs=cnn_model(num_words_rs,embedding_matrix_rs,sequence_length)
X_test_rs, y_test_rs=my_model(X_rs, y_rs,model_rs)

In [ ]:
size_of_data_rs= len(X_rs)
rs_result =predict_result(X_test_rs,y_test_rs, 'Random Swap',size_of_data_rs,model_rs)
print(rs_result)


Random Insertion

In [ ]:
ri_text=[]
ri = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    ri_text.append(ri.random_insertion(line))
print(len(ri_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

In [ ]:
ri_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
ri_text_combine= df['text'].tolist()
ri_bool_raiting=[]
for i in df['bool_raiting']:
  ri_bool_raiting.append(i)
print(ri_bool_raiting)

for i in ri_text:
  ri_text_combine.append(i)
  ri_bool_raiting.append(minor_class)
  #count+=1


In [ ]:
columns=['text','bool_raiting']
df_ri_synth = pd.DataFrame(columns=columns)
df_ri_synth = df_ri_synth.fillna(0)
df_ri_synth['text']=ri_text_combine
df_ri_synth['bool_raiting']=ri_bool_raiting

In [ ]:
df_ri_synth.head(5)

In [ ]:
df_ri_synth=shuffle_data(df_ri_synth)

In [ ]:
#Original Data
X_ri,y_ri,embedding_matrix_ri,num_words_ri =embedding(df_ri_synth['text'],df_ri_synth['bool_raiting'].values,sequence_length)#original data

In [ ]:
model_ri=cnn_model(num_words_ri,embedding_matrix_ri,sequence_length)
X_test_ri, y_test_ri=my_model(X_ri, y_ri,model_ri)

In [ ]:
size_of_data_ri= len(X_ri)
ri_result =predict_result(X_test_ri,y_test_ri, 'Random Insertion',size_of_data_ri,model_ri)
print(ri_result)


#END